## KarpovCourses. Продуктовая аналитика 
### Сегментация клиентов в Python 
 
### RFM-анализ

In [1]:
import pandas as pd
import numpy as np


# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'svg'")

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

In [2]:
orders = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-pozdniakov/product_analytics/RFM_ht_data.csv',sep=',')

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
orders['InvoiceDate'] = pd.to_datetime(orders['InvoiceDate'])
orders['InvoiceNo'] = orders['InvoiceDate'].apply(str)
orders['CustomerCode'] = orders['CustomerCode'].apply(str)

In [4]:
orders.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [5]:
orders.groupby('CustomerCode', as_index=False).agg({'InvoiceNo':'count'}).sort_values('InvoiceNo', ascending=False)

,CustomerCode,InvoiceNo
89388,19057820,204
44594,13215452,113
10347,13032521,106
97077,19080880,99
119951,99003061,90
...,...,...
58910,13272861,1
58911,13272871,1
58913,13272875,1
58914,13272878,1


In [6]:
last_date = orders.InvoiceDate.max()

In [8]:
orders.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,2020-09-01 00:00:00,19067290,2020-09-01,1716.00
1,2020-09-01 00:00:00,13233933,2020-09-01,1489.74
2,2020-09-01 00:00:00,99057968,2020-09-01,151.47
3,2020-09-01 00:00:00,80007276,2020-09-01,146.72
4,2020-09-01 00:00:00,13164076,2020-09-01,104.00


In [10]:

rfmTable = orders.groupby('CustomerCode', as_index=False).agg({'InvoiceDate': lambda x: (last_date - x.max()).days,                                  'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()})

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)

rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

In [11]:
rfmSegmentation = rfmTable

In [12]:

def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [14]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [15]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str)                             + rfmSegmentation.F_Quartile.map(str)                             + rfmSegmentation.M_Quartile.map(str)

In [25]:
rfmSegmentation.query('M_Quartile == 4').groupby('CustomerCode', as_index=False).agg({'monetary_value':'sum'}) \
    .sort_values('monetary_value', ascending=False)

,CustomerCode,monetary_value
28575,70024801,765.00
22974,19071804,765.00
2309,13036504,765.00
2376,13038952,765.00
26777,35039364,765.00
...,...,...
2936,13052590,9.50
17548,19006298,5.00
8002,13164125,2.64
27280,35076038,0.00


In [26]:
rfmSegmentation.query('F_Quartile == 1').groupby('CustomerCode', as_index=False).agg({'frequency':'sum'}) \
    .sort_values('frequency', ascending=False)

,CustomerCode,frequency
20611,19057820,204
10870,13215452,113
2616,13032521,106
22169,19080880,99
26868,99003061,90
...,...,...
13988,13271576,4
13989,13271577,4
13991,13271618,4
13992,13271677,4


In [27]:
rfmSegmentation.query('R_Quartile == 2').groupby('CustomerCode', as_index=False).agg({'recency':'max'}) \
    .sort_values('recency', ascending=False)

,CustomerCode,recency
17277,13280815,8
14209,13234867,8
14179,13234527,8
14173,13234312,8
14165,13234215,8
...,...,...
20331,18116243,3
12875,13216509,3
20334,18116543,3
27796,19088392,3


In [29]:
rfmSegmentation.query('R_Quartile == 1 and F_Quartile == 1 and M_Quartile == 1') \
        .nunique()

CustomerCode      1609
recency              8
frequency           16
monetary_value    1607
R_Quartile           1
F_Quartile           1
M_Quartile           1
RFMClass             1
dtype: int64

In [ ]:
rfmSegmentation.query('R_Quartile == 3 and F_Quartile == 1 and M_Quartile == 1') \
        .nunique()

In [32]:
rfmSegmentation.groupby('RFMClass', as_index=False) \
        .agg({'CustomerCode':'nunique'}) \
        .sort_values('CustomerCode', ascending=False)

,RFMClass,CustomerCode
63,444,10624
0,111,9705
62,443,6729
47,344,6593
16,211,5847
...,...,...
55,424,63
19,214,60
3,114,60
35,314,33


In [18]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [16]:
pd.crosstab(index = rfmSegmentation.R_Quartile, columns = rfmSegmentation.F_Quartile)

F_Quartile,1,2,3,4
R_Quartile,,,,
1,14433,4671,5795,6074
2,9634,5889,8633,10399
3,3091,3748,7734,13263
4,542,1311,5415,23101


In [ ]:
rfm_table = rfmSegmentation.pivot_table(
                        index='R_Quartile', 
                        columns='F_Quartile', 
                        values='monetary_value', 
                        aggfunc=np.median).applymap(int)
sns.heatmap(rfm_table, cmap="YlGnBu", annot=True, fmt=".0f", linewidths=4.15, annot_kws={"size": 10},yticklabels=4);